<a href="https://colab.research.google.com/github/HecVelaz/Codigos-de-Algoritmos--Ejemplos-variados/blob/main/Prediccion_de_Temperatura/codigoColab/Prediccion_de_temperatura_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nombre:Héctor Velázquez



In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [3]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 3.3 MB/s eta 0:00:00


In [4]:
import optuna


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [7]:
tf.config.list_physical_devices('GPU')
tf.config.experimental.enable_op_determinism()

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score
SEED = 7

In [9]:
# Imprimir los dispositivos físicos disponibles (GPU)
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
# Cargar el conjunto de datos
dataset = pd.read_csv("Temp_Asu20092021.csv")
datos=dataset

In [11]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37984 entries, 0 to 37983
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   37984 non-null  int64  
 1   Fecha        37984 non-null  object 
 2   Temperatura  37984 non-null  float64
 3   Humedad      37984 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.2+ MB


In [12]:
datos.head()

,Unnamed: 0,Fecha,Temperatura,Humedad
0,0,2009-01-01 00:00:00,26.749483,69.609451
1,1,2009-01-01 03:00:00,25.956114,78.225166
2,2,2009-01-01 06:00:00,23.707357,93.551756
3,3,2009-01-01 09:00:00,23.726823,93.394817
4,4,2009-01-01 12:00:00,29.606379,72.155839


In [ ]:
valores_faltantes_Fecha = datos['Fecha'].isnull().sum()
valores_faltantes_Fecha

0

In [ ]:
valores_faltantes_Temperatura = datos['Temperatura'].isnull().sum()
valores_faltantes_Temperatura

0

In [ ]:
valores_faltantes_Humedad = datos['Humedad'].isnull().sum()
valores_faltantes_Humedad

0

No hay valores faltantes en las columnas de fecha, temperatura y humedad

In [ ]:
valores_faltantes_Fecha = datos['Fecha'].isna().sum()
valores_faltantes_Fecha

0

In [ ]:
valores_faltantes_Temperatura = datos['Temperatura'].isna().sum()
valores_faltantes_Temperatura

0

In [ ]:
valores_faltantes_Humedad = datos['Humedad'].isna().sum()
valores_faltantes_Humedad

0

In [13]:
datos['Fecha'] = pd.to_datetime(datos['Fecha'])



In [14]:
datos.set_index('Fecha', inplace=True)


In [15]:
datos = datos[datos.index.year >= 2019]

In [16]:
datos.dropna(inplace=True)

<ipython-input-16-8df6119a9e81>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos.dropna(inplace=True)


In [17]:
datos_interpolados = datos.resample('1H').interpolate(method='spline', order=3)
datos_interpolados.dropna(inplace=True)

In [18]:
# CALCULAR EL MÁXIMO DE CADA DÍA
max_temperaturas_diarias = datos_interpolados.resample('D').max()

In [19]:
# Aplicar corte de cuartil a las temperaturas máximas diarias
cuartiles = max_temperaturas_diarias['Temperatura'].quantile([0.25, 0.5, 0.75])

In [20]:
# Definir categorías basadas en los cuartiles
umbral_frio = cuartiles[0.25]
umbral_bueno = cuartiles[0.5]
umbral_caliente = cuartiles[0.75]
print(cuartiles)
datos_interpolados['Max_Temperatura_Dia'] = datos_interpolados.groupby(datos_interpolados.index.date)['Temperatura'].transform('max')


0.25    27.066809
0.50    31.893574
0.75    36.407532
Name: Temperatura, dtype: float64


In [21]:
# Asignar cada observación de temperatura a su categoría correspondiente
datos_interpolados['Categoria_Temperatura'] = pd.cut(datos_interpolados['Max_Temperatura_Dia'], bins=[-float('inf'), umbral_frio, umbral_bueno, umbral_caliente, float('inf')],
                                      labels=['Frío', 'Bueno', 'Caliente', 'Muy Caliente'])

In [22]:
# Desplazar para obtener la temperatura del siguiente día
datos_interpolados['Temperatura_Siguiente_Dia'] = datos_interpolados['Categoria_Temperatura'].shift(-24)

In [23]:
# USAR ONE HOT ENCODING
datos_codificados = pd.get_dummies(datos_interpolados, columns=['Categoria_Temperatura','Temperatura_Siguiente_Dia'])



In [25]:
datos_codificados.head()

,Unnamed: 0,Temperatura,Humedad,Max_Temperatura_Dia,Categoria_Temperatura_Frío,Categoria_Temperatura_Bueno,Categoria_Temperatura_Caliente,Categoria_Temperatura_Muy Caliente,Temperatura_Siguiente_Dia_Frío,Temperatura_Siguiente_Dia_Bueno,Temperatura_Siguiente_Dia_Caliente,Temperatura_Siguiente_Dia_Muy Caliente
Fecha,,,,,,,,,,,,
2019-01-01 00:00:00,29216.000000,28.260936,66.311373,37.639718,0,0,0,1,0,0,0,1
2019-01-01 01:00:00,29216.333333,28.223146,66.523607,37.639718,0,0,0,1,0,0,0,1
2019-01-01 02:00:00,29216.666667,27.908703,67.385359,37.639718,0,0,0,1,0,0,0,1
2019-01-01 03:00:00,29217.000000,27.051418,68.882454,37.639718,0,0,0,1,0,0,0,1
2019-01-01 04:00:00,29217.333333,26.729742,70.296842,37.639718,0,0,0,1,0,0,0,1


In [26]:
# Definir características y variable objetivo
X = datos_codificados.iloc[:,[2,3,4,7]]  # Puedes probar diferentes combinaciones de características
y = datos_codificados.iloc[:,[8,11]]

In [27]:
X.head()

,Humedad,Max_Temperatura_Dia,Categoria_Temperatura_Frío,Categoria_Temperatura_Muy Caliente
Fecha,,,,
2019-01-01 00:00:00,66.311373,37.639718,0,1
2019-01-01 01:00:00,66.523607,37.639718,0,1
2019-01-01 02:00:00,67.385359,37.639718,0,1
2019-01-01 03:00:00,68.882454,37.639718,0,1
2019-01-01 04:00:00,70.296842,37.639718,0,1


In [28]:
y.head()

,Temperatura_Siguiente_Dia_Frío,Temperatura_Siguiente_Dia_Muy Caliente
Fecha,,
2019-01-01 00:00:00,0,1
2019-01-01 01:00:00,0,1
2019-01-01 02:00:00,0,1
2019-01-01 03:00:00,0,1
2019-01-01 04:00:00,0,1


In [29]:
scaler = MinMaxScaler()
X_escalado = scaler.fit_transform(X)


In [30]:
# Dividir datos en conjuntos de entrenamiento y validación
indices_entrenamiento = (datos_interpolados.index.year <= 2019)
indices_validacion = ((datos_interpolados.index.year >= 2020) & (datos_interpolados.index.year <= 2020))

In [31]:
# Dividir datos en conjuntos de entrenamiento y validación
X_entrenamiento, X_validacion = X_escalado[indices_entrenamiento], X_escalado[indices_validacion]
y_entrenamiento, y_validacion = y[indices_entrenamiento], y[indices_validacion]

In [32]:
len_y = len(y)
len_X_escalado = len(X_escalado)

print("Longitud de y:", len_y)
print("Longitud de X_escalado:", len_X_escalado)

if len_y == len_X_escalado:
    print("La cantidad de filas en y es igual a la cantidad de filas en X_escalado.")
else:
    print("La cantidad de filas en y no es igual a la cantidad de filas en X_escalado.")

Longitud de y: 26302
Longitud de X_escalado: 26302
La cantidad de filas en y es igual a la cantidad de filas en X_escalado.


In [33]:
print(X_entrenamiento.shape, X_validacion.shape)
print(y_entrenamiento.shape, y_validacion.shape)


(8760, 4) (8784, 4)
(8760, 2) (8784, 2)


In [34]:
def objetivo(trial):
    # Definir hiperparámetros a ajustar
    num_capas = 1
    tf.random.set_seed(SEED)

    num_unidades = trial.suggest_categorical('num_unidades', [32,64,128]) #cantidad de capas ocultas de la red
    tasa_aprendizaje = trial.suggest_categorical('tasa_aprendizaje', [1e-2, 1e-3, 1e-4])

    # Definir la arquitectura del modelo
    modelo = Sequential()
    modelo.add(Dense(num_unidades, activation='relu', input_shape=(X.shape[1],)))
    for _ in range(num_capas - 1):
        modelo.add(Dense(num_unidades, activation='relu'))
    modelo.add(Dense(2, activation='softmax'))

    # Compilar el modelo
    modelo.compile(optimizer=Adam(learning_rate=tasa_aprendizaje),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Entrenar el modelo
    modelo.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion), epochs=30, batch_size=256, verbose=0, shuffle=False)

    # Evaluar el modelo en el conjunto de validación
    _, val_acc = modelo.evaluate(X_validacion, y_validacion, verbose=0)

    return val_acc

In [35]:
# Realizar la optimización de hiperparámetros usando Optuna
estudio = optuna.create_study(direction='maximize')
estudio.optimize(objetivo, n_trials=50)

[I 2024-03-30 21:22:35,768] A new study created in memory with name: no-name-161fa833-cb38-4164-b6bf-2cc567d66332
[I 2024-03-30 21:22:47,494] Trial 0 finished with value: 0.6785063743591309 and parameters: {'num_unidades': 64, 'tasa_aprendizaje': 0.0001}. Best is trial 0 with value: 0.6785063743591309.
[I 2024-03-30 21:22:55,276] Trial 1 finished with value: 0.8032786846160889 and parameters: {'num_unidades': 32, 'tasa_aprendizaje': 0.0001}. Best is trial 1 with value: 0.8032786846160889.
[I 2024-03-30 21:23:01,388] Trial 2 finished with value: 0.7339481115341187 and parameters: {'num_unidades': 32, 'tasa_aprendizaje': 0.001}. Best is trial 1 with value: 0.8032786846160889.
[I 2024-03-30 21:23:12,847] Trial 3 finished with value: 0.6912568211555481 and parameters: {'num_unidades': 64, 'tasa_aprendizaje': 0.001}. Best is trial 1 with value: 0.8032786846160889.
[I 2024-03-30 21:23:24,141] Trial 4 finished with value: 0.7223360538482666 and parameters: {'num_unidades': 32, 'tasa_aprendiza

In [36]:
# Obtener los mejores hiperparámetros
mejor_num_capas = 1
mejor_num_unidades = estudio.best_params['num_unidades']
mejor_tasa_aprendizaje = estudio.best_params['tasa_aprendizaje']

In [37]:
print("Mejores Hiperparámetros:")
print("Número de Capas:", mejor_num_capas)
print("Número de Unidades:", mejor_num_unidades)
print("Tasa de Aprendizaje:", mejor_tasa_aprendizaje)

Mejores Hiperparámetros:
Número de Capas: 1
Número de Unidades: 32
Tasa de Aprendizaje: 0.0001


In [38]:
# Entrenar el modelo final usando los mejores hiperparámetros
modelo_final = Sequential()
modelo_final.add(Dense(mejor_num_unidades, activation='relu', input_shape=(X_entrenamiento.shape[1],)))
for _ in range(mejor_num_capas - 1):
    modelo_final.add(Dense(mejor_num_unidades, activation='relu'))
modelo_final.add(Dense(2, activation='softmax'))

modelo_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=mejor_tasa_aprendizaje),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

In [39]:
# Dividir datos en conjuntos de entrenamiento y prueba
indices_entrenamiento_final = (datos_codificados.index.year < 2020)
indices_prueba_final = (datos_codificados.index.year >= 2021)

In [40]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_entrenamiento_final, X_prueba_final = X_escalado[indices_entrenamiento_final], X_escalado[indices_prueba_final]
y_entrenamiento_final, y_prueba_final = y[indices_entrenamiento_final], y[indices_prueba_final]


In [41]:
# Entrenar modelo final
modelo_final.fit(X_entrenamiento_final, y_entrenamiento_final, validation_data=(X_prueba_final, y_prueba_final), epochs=100, batch_size=256, verbose=1, shuffle=False)


Epoch 1/100
35/35 [==============================] - 1s 12ms/step - loss: 0.3928 - accuracy: 0.7150 - val_loss: 0.3119 - val_accuracy: 0.7045
Epoch 2/100
35/35 [==============================] - 0s 5ms/step - loss: 0.3884 - accuracy: 0.7164 - val_loss: 0.3087 - val_accuracy: 0.7043
Epoch 3/100
35/35 [==============================] - 0s 6ms/step - loss: 0.3840 - accuracy: 0.7163 - val_loss: 0.3054 - val_accuracy: 0.7044
Epoch 4/100
35/35 [==============================] - 0s 5ms/step - loss: 0.3796 - accuracy: 0.7163 - val_loss: 0.3022 - val_accuracy: 0.7043
Epoch 5/100
35/35 [==============================] - 0s 5ms/step - loss: 0.3753 - accuracy: 0.7162 - val_loss: 0.2991 - val_accuracy: 0.7034
Epoch 6/100
35/35 [==============================] - 0s 5ms/step - loss: 0.3710 - accuracy: 0.7152 - val_loss: 0.2959 - val_accuracy: 0.7008
Epoch 7/100
35/35 [==============================] - 0s 5ms/step - loss: 0.3667 - accuracy: 0.7146 - val_loss: 0.2928 - val_accuracy: 0.6995
Epoch 8/100


In [42]:
# Evaluar en conjunto de prueba
_, test_acc = modelo_final.evaluate(X_prueba_final, y_prueba_final, verbose=0)
print("Accuracy en conjunto de prueba:", test_acc)

Accuracy en conjunto de prueba: 0.7724366188049316


In [43]:
# Calcular otras métricas
y_pred = modelo_final.predict(X_prueba_final)
y_pred_classes = np.argmax(y_pred, axis=1)

274/274 [==============================] - 0s 1ms/step


In [44]:
# Convertir y_prueba_final de representación one-hot a clases numéricas
y_prueba_final_clases = np.argmax(y_prueba_final.values, axis=1)

# Calcular las métricas utilizando las clases numéricas
precision = precision_score(y_prueba_final_clases, y_pred_classes, average='weighted')
recall = recall_score(y_prueba_final_clases, y_pred_classes, average='weighted')
f1 = f1_score(y_prueba_final_clases, y_pred_classes, average='weighted')

In [45]:
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.8451149376381033
Recall: 0.7724366293674355
F1 Score: 0.7909604509701612
